# 전력 수요량 예측 모델링_2
##### 20191022 - 
##### ㅇ
Train 데이터로 모델을 학습, 검증 후 Test로 예측합니다.


- 보간법으로 결측치 채워보기

In [7]:
# import package
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns 

from datetime import datetime, timedelta

from sklearn.cluster import KMeans

%matplotlib inline

## 1. 데이터 불러오기

In [28]:
train = pd.read_csv("11회 data/train.csv")
train['Time'] = pd.to_datetime(train['Time'])
#train = train.set_index('Time')

In [29]:
train_ex = train.copy()  # 원본 데이터 보존을 위해 복사
#train_ex = train_ex.set_index('Time')
train_ex.shape

(16909, 1301)

In [12]:
# 모든 행이 NaN인 세대 수 1301-1293=8세대
NaN_list = set(train_ex.dropna(how='all', axis=1).keys())
set(train_ex.keys()).difference(NaN_list)

NaN_X = set(train_ex.keys()).difference(NaN_list)

In [14]:
first_notnull_2 = []  # 빈 list

for i in train_ex.keys()[1:]:
    if i not in NaN_X:
       _ = train_ex[i]
       first_notnull_2.append(_[_.isnull() == False].index[0])
    else:
        first_notnull_2.append(0)
        
df_notnull_2 = pd.DataFrame(first_notnull_2, list(train_ex.keys()[1:]))
df_notnull_2.columns = ['not_blank_num']

In [16]:
train_ex['Time']

0       2016-07-26 11:00:00
1       2016-07-26 12:00:00
2       2016-07-26 13:00:00
3       2016-07-26 14:00:00
4       2016-07-26 15:00:00
                ...        
16904   2018-06-30 19:00:00
16905   2018-06-30 20:00:00
16906   2018-06-30 21:00:00
16907   2018-06-30 22:00:00
16908   2018-06-30 23:00:00
Name: Time, Length: 16909, dtype: datetime64[ns]

In [18]:
train_ex.head()

,Time,X692,X1272,X553,X1299,X4,X598,X1003,X1010,X1216,...,X1164,X656,X1287,X324,X767,X1271,X1288,X58,X1033,X1163
0,2016-07-26 11:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2016-07-26 12:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2016-07-26 13:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2016-07-26 14:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2016-07-26 15:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
# index Time으로 고정
#train_ex = train_ex.set_index('Time')

start_num = df_notnull_2.loc['X692'][0]
train_ex[['X692']][start_num:]

,X692
Time,
2018-02-15 00:00:00,0.150
2018-02-15 01:00:00,0.422
2018-02-15 02:00:00,0.314
2018-02-15 03:00:00,0.309
2018-02-15 04:00:00,0.327
...,...
2018-06-30 19:00:00,0.271
2018-06-30 20:00:00,0.341
2018-06-30 21:00:00,0.535


### pandas로 결측값 보간(interpolation)
: train에서 20%(260세대) 랜덤으로 추출해서 csv로 내보내기

1. 시계열데이터의 값에 선형으로 비례하는 방식으로 결측값 보간
2. 시계열 날짜 index를 기준으로 결측값 보간

In [75]:
tsvar = train_ex[['X692']][start_num:]

In [52]:
# 1. 시계열데이터의 값에 선형으로 비례하는 방식으로 결측값 보간
ts_inter_1 = tsvar.interpolate()

# 2. 시계열 날짜 index를 기준으로 결측값 보간
ts_inter_2 = tsvar.interpolate(method='time')

# 3. DataFrame 값에 선형으로 비례하는 방식으로 결측값 보간
ts_inter_3 = tsvar.interpolate(method='values')

In [65]:
a = pd.concat([tsvar, ts_inter_1, ts_inter_2, ts_inter_3],
         axis=1, keys=['method1', 'method2', 'method3'])
a.loc['2018-03-01 2:00':'2018-03-02 2:00']

,method1,method2,method3
,X692,X692,X692
Time,,,
2018-03-01 02:00:00,NaN,0.2940,0.2940
2018-03-01 03:00:00,NaN,0.2870,0.2870
2018-03-01 04:00:00,NaN,0.2800,0.2800
2018-03-01 05:00:00,NaN,0.2730,0.2730
2018-03-01 06:00:00,NaN,0.2660,0.2660
2018-03-01 07:00:00,NaN,0.2590,0.2590
2018-03-01 08:00:00,NaN,0.2520,0.2520
2018-03-01 09:00:00,NaN,0.2450,0.2450


### 랜덤으로 train에서 20%(260세대)추출

In [72]:
import random

In [201]:
train_ex2 = train.copy()
train_ex2 = train_ex2.set_index('Time')

household = list(train.keys()[1:])  # Time 제외한 세대명
random_house = random.sample(household, 260)

In [203]:
### Method 1
# 전체 Time 데이터
Raw_Time_df = train[['Time']]

# Time변수만 df로 추출
Time_df = train_ex2[:1]

#Raw_Time_df = Raw_Time_df.set_index('Time')
result = Raw_Time_df

for k in random_house:
    # randomhouse에 해당하는 k세대만큼 for문 동작
    start_num = df_notnull_2.loc[k][0]
    tsvar = train_ex2[[k]][start_num:]
    
    # Time 인덱스를 column값으로 보내기
    tsvar.reset_index(level=['Time'], inplace=True)
    
    ts_inter_1 = tsvar.interpolate()  # method 1
    ts_inter_1 = ts_inter_1.set_index(tsvar.index)
    result = pd.merge(result, ts_inter_1, how='left',
                      on='Time')
    
    #ts_inter_2 = tsvar.interpolate(method='time')
    #ts_inter_3 = tsvar.interpolate(method='values')

In [204]:
# Method 1 : csv로 변환
result.to_csv('interpolation_1.csv', index=False)

In [212]:
### Method 2
# 전체 Time 데이터
Raw_Time_df = train[['Time']]

# Time변수만 df로 추출
Time_df = train_ex2[:1]

#Raw_Time_df = Raw_Time_df.set_index('Time')
result = Raw_Time_df


for k in random_house:
    # randomhouse에 해당하는 k세대만큼 for문 동작
    start_num = df_notnull_2.loc[k][0]
    tsvar = train_ex2[[k]][start_num:]
    
    ts_inter_2 = tsvar.interpolate(method='time')  # method 2
    
    # Time 인덱스를 column값으로 보내기
    #tsvar.reset_index(level=['Time'], inplace=True)
    
    ts_inter_2 = ts_inter_2.set_index(tsvar.index)
    result = pd.merge(result, ts_inter_2, how='left',
                      on='Time')
    
    #ts_inter_2 = tsvar.interpolate(method='time')
    #ts_inter_3 = tsvar.interpolate(method='values')

In [217]:
# Method 2 : csv로 변환
result.to_csv('interpolation_2.csv', index=False)

In [213]:
result

,Time,X1472,X115,X1226,X273,X339,X774,X857,X1075,X1061,...,X933,X1238,X394,X829,X432,X1334,X1416,X803,X45,X1205
0,2016-07-26 11:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2016-07-26 12:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2016-07-26 13:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2016-07-26 14:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2016-07-26 15:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16904,2018-06-30 19:00:00,0.331,1.325,0.356,0.329,0.330,0.756,0.694,0.172,1.010,...,0.222,0.285,0.262,0.644,0.277,0.689,0.835,0.879,NaN,0.281
16905,2018-06-30 20:00:00,0.404,1.218,0.346,0.335,0.387,0.445,0.912,0.734,0.681,...,0.222,0.301,0.287,0.720,0.300,0.832,0.435,0.408,NaN,0.245
16906,2018-06-30 21:00:00,0.379,1.100,0.302,0.291,0.446,0.386,0.844,0.934,0.548,...,0.222,0.297,0.292,0.694,0.305,0.484,0.487,0.636,NaN,0.483
16907,2018-06-30 22:00:00,0.553,1.094,0.233,0.278,0.385,0.393,0.840,0.715,0.589,...,0.222,0.406,0.258,0.582,0.233,0.781,0.318,0.504,NaN,0.467


In [216]:
pd.Series(result['Time']).value_counts()

2017-09-18 20:00:00    1
2016-07-29 18:00:00    1
2017-02-21 10:00:00    1
2018-02-04 01:00:00    1
2017-05-27 20:00:00    1
                      ..
2016-11-03 03:00:00    1
2018-05-08 17:00:00    1
2018-06-25 13:00:00    1
2016-11-12 02:00:00    1
2017-04-30 05:00:00    1
Name: Time, Length: 16909, dtype: int64

### 기상 데이터 결측치 보간법으로 채우기


In [223]:
hour_weather = pd.read_csv("11회 data 과거 인천 기상예측/hourly_weather.csv",
                          encoding='cp949')
hour_weather.head()

,지점,일시,기온,강수량(mm),풍속(m/s),습도(per),적설(cm),날씨,전운량(10분위)
0,112,2016.7.20 0:00,25.7,NaN,1.3,98.0,NaN,박무,8.0
1,112,2016.7.20 1:00,25.6,NaN,1.4,98.0,NaN,NaN,NaN
2,112,2016.7.20 2:00,25.4,NaN,0.3,98.0,NaN,NaN,NaN
3,112,2016.7.20 3:00,24.9,NaN,1.1,98.0,NaN,박무,8.0
4,112,2016.7.20 4:00,24.7,NaN,0.7,98.0,NaN,박무,8.0


In [235]:
hour_weather.columns = ['place', 'Time', 'temp', 'rainfall', 'wind',
                      'hum', 'snowfall', 'weather', 'cloud']
weather_re = hour_weather[['Time', 'temp', 'hum']]

In [237]:
weather_re['Time'] = pd.to_datetime(weather_re['Time'])
weather_re.head()

C:\Users\huiyeon\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Time,temp,hum
0,2016-07-20 00:00:00,25.7,98.0
1,2016-07-20 01:00:00,25.6,98.0
2,2016-07-20 02:00:00,25.4,98.0
3,2016-07-20 03:00:00,24.9,98.0
4,2016-07-20 04:00:00,24.7,98.0


In [240]:
# 1. 시계열데이터의 값에 선형으로 비례하는 방식으로 결측값 보간
weather_inter_1 = weather_re.interpolate()

In [249]:
# 2. 시계열 날짜 index를 기준으로 결측값 보간
weather_re = weather_re.set_index('Time')
weather_inter_2 = weather_re.interpolate(method='time')

In [250]:
# 3. DataFrame 값에 선형으로 비례하는 방식으로 결측값 보간
weather_inter_3 = weather_re.interpolate(method='values')

In [257]:
weather_inter_1.to_csv('weather_inter_1.csv', index=False)
weather_inter_2.to_csv('weather_inter_2.csv', index=False)